In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


import torchvision
from torchvision import transforms


import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as TF

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from PIL import Image

# Necessary imports

from GLOBAL_VAR import *

/Users/josephallyndree/miniconda3/envs/python_rapids/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/josephallyndree/miniconda3/envs/python_rapids/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/josephallyndree/miniconda3/envs/python_rapids/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <95ED4CE3-7668-35AB-A545-2793CE3BB2EC> /Users/josephallyndree/miniconda3/envs/python_rapids/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Bruit gaussien

In [2]:
class AddGaussianNoise(object):
    def __init__(self, mean=0, std=0.01):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean


In [3]:
class ImageFolderWithRotations:
    def __init__(self, root, transform=None, target_transform=None):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.imgs = self._load_imgs()

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = self.loader(path)
        
        if self.transform is not None:
            img = self.transform(img)
        
        # Perform rotations
        rot_90 = transforms.functional.rotate(img, 90)
        rot_180 = transforms.functional.rotate(img, 180)
        rot_270 = transforms.functional.rotate(img, 270)
        
        # Convert PIL.Image to Tensor
        rot_90 = transforms.ToTensor()(rot_90)
        rot_180 = transforms.ToTensor()(rot_180)
        rot_270 = transforms.ToTensor()(rot_270)
        img = transforms.ToTensor()(img)
        
        # Return rotated images and their labels
        return [(img, target), (rot_90, target), (rot_180, target), (rot_270, target)]

    def _load_imgs(self):
        # Load original images
        data_noisy = ImageFolder(root=self.root, transform=self.transform, target_transform=self.target_transform)
        imgs = data_noisy.imgs

        # Add rotated images to the dataset
        new_imgs = []
        for i, (path, target) in enumerate(imgs):
            img = Image.open(path).convert('RGB')

            rot_90 = transforms.functional.rotate(img, 90)
            rot_180 = transforms.functional.rotate(img, 180)
            rot_270 = transforms.functional.rotate(img, 270)

            new_path_90 = path.replace('.jpg', '_rot90.jpg')
            new_path_180 = path.replace('.jpg', '_rot180.jpg')
            new_path_270 = path.replace('.jpg', '_rot270.jpg')

            rot_90.save(new_path_90)
            rot_180.save(new_path_180)
            rot_270.save(new_path_270)

            new_imgs.append((path, target))
            new_imgs.append((new_path_90, target))
            new_imgs.append((new_path_180, target))
            new_imgs.append((new_path_270, target))

        return new_imgs



In [4]:
import shutil
import os


shutil.copyfile(original_path, new_path)

os.rename(new_path, "C:\Users\Administrateur\Downloads\augmentation\data_augmented_001.zip" )

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (4281882314.py, line 4)

In [19]:
# Create the ImageFolderWithRotations instance
data_noisy_rot = ImageFolderWithRotations(root=r"C:\Users\Administrateur\Downloads\augmentation\data_augmented_001.zip\data", transform = AddGaussianNoise())


In [20]:
print(len(data_noisy_rot))
print(type(data_noisy_rot))

14400
<class '__main__.ImageFolderWithRotations'>


# Vérification si ça marche

In [21]:
new_data = torchvision.datasets.ImageFolder(root=r"C:\Users\Administrateur\Downloads\data_aug_001\data", transform=transforms.ToTensor())

In [22]:
ims = []
labs = []
for image, label in new_data:
    ims.append(image)
    labs.append(labs)

In [23]:
print(len(ims), len(labs))

14400 14400


In [ ]:
fig, axs = plt.subplots(1, len(ims[0:4]),  figsize=(20,10))

for i, image_tensor in enumerate(ims[0:4]):
    axs[i].imshow(image_tensor)

plt.show()

TypeError: Invalid shape (3, 100, 100) for image data

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6f211ba4-8439-4b4f-a72d-48759d094385' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>